In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib
import os

from scipy.stats import zscore
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline

from numpy import mean,std
from sklearn.model_selection import RepeatedKFold,cross_val_score,KFold,train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,MultiLabelBinarizer
from sklearn.metrics import mean_squared_error, mean_absolute_error,accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from keras.utils import normalize, to_categorical
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

import fnmatch
import os

In [3]:
def downsample(df:pd.DataFrame, label_col_name:str) -> pd.DataFrame:
    # find the number of observations in the smallest group
    nmin = df[label_col_name].value_counts().min()
    return (df
            # split the dataframe per group
            .groupby(label_col_name)
            # sample nmin observations from each group
            .apply(lambda x: x.sample(nmin))
            # recombine the dataframes
            .reset_index(drop=True)
            )


In [6]:

path_data = 'data-all/data-ecg-preprocessed/'
subjects = os.listdir(path_data)
subjects =[x for x in subjects if x.endswith(".csv")]

df = pd.DataFrame()

for data in subjects:
    
    data = pd.read_csv(path_data+data)
    df = df.append(data, ignore_index=True)
    
# df.to_csv('test.csv')    
#drop measures that are mostly Nan anyway
df.drop(['HRV_SDANN1_diff','HRV_SDNNI1_diff','HRV_SDANN2_diff','HRV_SDNNI2_diff','HRV_SDANN5_diff', \
         'HRV_SDNNI5_diff','HRV_ULF_diff','HRV_VLF_diff','HRV_LF_diff','HRV_VHF_diff', 'HRV_LFHF_diff', \
         'HRV_LFn_diff','HRV_MSE_diff','HRV_CMSE_diff','HRV_RCMSE_diff','HRV_SampEn_diff'],axis=1, inplace=True)  

#dropna based on most basic feature - if this is nan then all other would be 
df.dropna(subset=['HRV_MeanNN_diff', 'HRV_SDNN_diff','HRV_pNN50_diff', 'HRV_pNN20_diff','HR_DIFF'], inplace = True)

/tmp/ipykernel_10011/2376533322.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/tmp/ipykernel_10011/2376533322.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/tmp/ipykernel_10011/2376533322.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/tmp/ipykernel_10011/2376533322.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/tmp/ipykernel_10011/2376533322.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version.

In [8]:
def cond(row):
    if '+' in row['COND']:
        return 1
    elif '-' in row['COND']:
        return 0
    return None
df['COND_BIN'] = df.apply(lambda row: cond(row),axis=1)

In [9]:
df

,TIMESTAMP,ID,COND,IADS-ID,IAPS-ID,ANS-VALENCE,ANS-AROUSAL,ANS-TIME,EVENT,SUB-NAME,...,HRV_ApEn_diff,HRV_ShanEn_diff,HRV_FuzzyEn_diff,HRV_CD_diff,HRV_HFD_diff,HRV_KFD_diff,HRV_LZC_diff,HR_DIFF,IS_HR_ELEVATED,COND_BIN
0,1.580387e+09,444.0,PS-,105,3110,3.545,6.987,5.494,NaN,SUB444,...,0.000000,0.000000,0.514671,-0.216053,0.103447,0.485332,0.000000,0.987911,0.0,0.0
1,1.580387e+09,444.0,PS0,403,2695,5.004,6.924,4.540,NaN,SUB444,...,-0.045566,-0.250000,0.476315,-0.129926,-0.053318,-0.376849,-0.250000,0.990807,0.0,NaN
2,1.580387e+09,444.0,S0,361,None,4.423,3.860,4.824,NaN,SUB444,...,-0.193335,0.000000,0.857799,-0.106149,0.101229,0.649000,-0.250000,1.024919,1.0,NaN
3,1.580387e+09,444.0,P0,None,3310,4.141,3.824,1.317,NaN,SUB444,...,-0.370674,0.023557,0.115035,-0.035931,-0.434954,0.138542,-0.458163,0.960962,0.0,NaN
4,1.580387e+09,444.0,PS-,624,3195,4.318,7.867,2.461,NaN,SUB444,...,0.045566,-0.125000,0.187937,-0.130734,-0.385782,0.203936,-0.250000,1.041281,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9709,1.579783e+09,249.0,S+,355,None,6.779,6.056,1.15,NaN,SUB249,...,-0.262110,-0.106915,0.220776,-0.101061,0.558943,0.693883,0.620080,1.004924,1.0,1.0
9710,1.579783e+09,249.0,S+,716,None,7.503,6.388,2.883,NaN,SUB249,...,-0.047947,0.000000,-0.158513,0.048871,-0.378279,-1.157207,0.000000,1.033457,1.0,1.0
9711,1.579783e+09,249.0,P0,None,4274,8.005,6.003,1.783,NaN,SUB249,...,-0.051530,-0.260761,0.183797,-0.090309,-0.496785,0.199017,-0.221173,1.023592,1.0,NaN
9712,1.579783e+09,249.0,S+,254,None,7.489,5.795,4.883,NaN,SUB249,...,-0.180123,-0.360297,0.359673,-0.027996,-0.411804,-0.489137,0.120949,0.953630,0.0,1.0


In [10]:
df.dropna(subset=['COND_BIN'], inplace = True)

In [11]:
df

,TIMESTAMP,ID,COND,IADS-ID,IAPS-ID,ANS-VALENCE,ANS-AROUSAL,ANS-TIME,EVENT,SUB-NAME,...,HRV_ApEn_diff,HRV_ShanEn_diff,HRV_FuzzyEn_diff,HRV_CD_diff,HRV_HFD_diff,HRV_KFD_diff,HRV_LZC_diff,HR_DIFF,IS_HR_ELEVATED,COND_BIN
0,1.580387e+09,444.0,PS-,105,3110,3.545,6.987,5.494,NaN,SUB444,...,0.000000,0.000000,0.514671,-0.216053,0.103447,0.485332,0.000000,0.987911,0.0,0.0
4,1.580387e+09,444.0,PS-,624,3195,4.318,7.867,2.461,NaN,SUB444,...,0.045566,-0.125000,0.187937,-0.130734,-0.385782,0.203936,-0.250000,1.041281,1.0,0.0
5,1.580387e+09,444.0,P-,None,3080,3.542,5.753,5.534,NaN,SUB444,...,-0.184839,0.000000,0.432634,-0.122730,-0.262350,-0.020525,-0.250000,0.984736,0.0,0.0
6,1.580388e+09,444.0,P+,None,8090,5.069,3.450,5.700,NaN,SUB444,...,-0.356476,0.000000,0.669768,0.145454,-0.174518,-0.018412,-0.250000,1.019500,1.0,1.0
8,1.580388e+09,444.0,S+,224,None,6.185,6.584,1.012,NaN,SUB444,...,-0.259402,0.094816,0.364205,-0.108071,-0.081926,0.603164,-0.057366,1.037793,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9707,1.579783e+09,249.0,S+,111,None,8.391,3.018,2.333,NaN,SUB249,...,-0.015206,-0.260761,0.877922,-0.198252,0.208363,0.672145,0.348126,0.941021,0.0,1.0
9709,1.579783e+09,249.0,S+,355,None,6.779,6.056,1.15,NaN,SUB249,...,-0.262110,-0.106915,0.220776,-0.101061,0.558943,0.693883,0.620080,1.004924,1.0,1.0
9710,1.579783e+09,249.0,S+,716,None,7.503,6.388,2.883,NaN,SUB249,...,-0.047947,0.000000,-0.158513,0.048871,-0.378279,-1.157207,0.000000,1.033457,1.0,1.0
9712,1.579783e+09,249.0,S+,254,None,7.489,5.795,4.883,NaN,SUB249,...,-0.180123,-0.360297,0.359673,-0.027996,-0.411804,-0.489137,0.120949,0.953630,0.0,1.0


In [12]:
#downsample:
df_downsampled = downsample(df, 'COND_BIN')

In [13]:
df_downsampled.shape

(5490, 73)

In [14]:
data = df_downsampled[['HRV_MeanNN_diff', 'HRV_SDNN_diff', 'HRV_RMSSD_diff', 'HRV_SDSD_diff',
       'HRV_CVNN_diff', 'HRV_CVSD_diff', 'HRV_MedianNN_diff', 'HRV_MadNN_diff', 'HRV_MCVNN_diff',
       'HRV_IQRNN_diff', 'HRV_pNN50_diff', 'HRV_pNN20_diff', 'HRV_HTI_diff', 'HRV_TINN_diff',
       'HRV_SD1_diff', 'HRV_SD2_diff', 'HRV_SD1SD2_diff',
       'HRV_CSI_diff', 'HRV_CVI_diff', 'HRV_CSI_Modified_diff', 'HRV_PIP_diff', 'HRV_IALS_diff',
       'HRV_PSS_diff', 'HRV_GI_diff', 'HRV_SI_diff', 'HRV_AI_diff', 'HRV_PI_diff', 'HRV_C1d_diff',
       'HRV_C1a_diff', 'HRV_SD1d_diff', 'HRV_SD1a_diff', 'HRV_C2d_diff', 'HRV_C2a_diff', 'HRV_SD2d_diff',
       'HRV_SD2a_diff', 'HRV_Cd_diff', 'HRV_Ca_diff', 'HRV_SDNNd_diff', 'HRV_SDNNa_diff',
       'HRV_DFA_alpha1_diff', 'HRV_DFA_alpha1_ExpRange_diff', 'HRV_DFA_alpha1_ExpMean_diff',
       'HRV_DFA_alpha1_DimRange_diff', 'HRV_DFA_alpha1_DimMean_diff', 'HRV_ApEn_diff',
       'HRV_ShanEn_diff', 'HRV_FuzzyEn_diff', 'HRV_CD_diff', 'HRV_HFD_diff',
       'HRV_KFD_diff', 'HRV_LZC_diff','HR_DIFF','IS_HR_ELEVATED']]

# data = df_downsampled[['HRV_MeanNN', 'HRV_SDNN','HRV_pNN50', 'HRV_pNN20','HR_DIFF','IS_HR_ELEVATED']]
labels = df_downsampled[['AROUSAL_CAT','VALENCE_CAT','AROUSAL_VALENCE_CAT','COND_BIN']]

In [15]:
pd.set_option('display.max_columns', 100)
data.head(5)

,HRV_MeanNN_diff,HRV_SDNN_diff,HRV_RMSSD_diff,HRV_SDSD_diff,HRV_CVNN_diff,HRV_CVSD_diff,HRV_MedianNN_diff,HRV_MadNN_diff,HRV_MCVNN_diff,HRV_IQRNN_diff,HRV_pNN50_diff,HRV_pNN20_diff,HRV_HTI_diff,HRV_TINN_diff,HRV_SD1_diff,HRV_SD2_diff,HRV_SD1SD2_diff,HRV_CSI_diff,HRV_CVI_diff,HRV_CSI_Modified_diff,HRV_PIP_diff,HRV_IALS_diff,HRV_PSS_diff,HRV_GI_diff,HRV_SI_diff,HRV_AI_diff,HRV_PI_diff,HRV_C1d_diff,HRV_C1a_diff,HRV_SD1d_diff,HRV_SD1a_diff,HRV_C2d_diff,HRV_C2a_diff,HRV_SD2d_diff,HRV_SD2a_diff,HRV_Cd_diff,HRV_Ca_diff,HRV_SDNNd_diff,HRV_SDNNa_diff,HRV_DFA_alpha1_diff,HRV_DFA_alpha1_ExpRange_diff,HRV_DFA_alpha1_ExpMean_diff,HRV_DFA_alpha1_DimRange_diff,HRV_DFA_alpha1_DimMean_diff,HRV_ApEn_diff,HRV_ShanEn_diff,HRV_FuzzyEn_diff,HRV_CD_diff,HRV_HFD_diff,HRV_KFD_diff,HRV_LZC_diff,HR_DIFF,IS_HR_ELEVATED
0,19.470588,-30.747954,-43.020672,-44.796249,-0.048840,-0.067922,26.0,-25.2042,-0.040480,-46.00,-11.764706,-5.882353,-0.850000,-70.3125,-31.675731,-35.770654,-0.221340,0.521113,-1.084966,-147.071197,0.117647,0.145833,0.123077,8.823741,9.900597,7.765811,-35.833333,-0.121259,0.121259,-27.654167,-15.153842,0.613662,-0.613662,-11.562409,-39.937315,0.376927,-0.376927,-18.964620,-29.622309,-0.177675,-2.111304,-0.985338,-0.368685,0.441846,-0.222449,-0.117647,0.714557,0.343668,0.295723,0.062262,0.480878,0.965309,0.0
1,21.627451,-46.367369,-64.810275,-66.914434,-0.066040,-0.092052,0.0,-1.4826,-0.002045,-1.00,-17.647059,-28.627451,0.350000,-242.1875,-47.315650,-49.365699,-0.331409,0.746953,-1.442776,-184.395343,0.011765,0.026786,-0.009091,9.220606,9.327372,9.119107,-19.642857,0.041904,-0.041904,-37.714705,-28.547534,0.232908,-0.232908,-26.002987,-42.639506,0.129314,-0.129314,-31.648298,-36.191656,0.222870,-2.128447,-0.629624,-0.562097,0.230111,-0.485829,0.054722,0.537426,0.013139,-0.353920,0.305769,-0.400796,0.962086,0.0
2,1.414286,-12.483219,-40.843517,-43.306894,-0.016845,-0.054832,1.0,15.5673,0.020662,18.75,-6.190476,-16.666667,2.000000,-125.0000,-30.622599,-7.818709,-0.448592,1.634089,-0.477924,316.335854,0.038095,0.076923,0.013889,14.285714,15.221071,13.356390,-7.692308,0.161617,-0.161617,-20.856503,-22.097199,0.114618,-0.114618,-1.040951,-10.237762,0.089205,-0.089205,-8.515709,-13.447674,0.703082,-4.325192,-0.937332,-1.830648,0.976115,-0.127097,0.033798,0.021048,-0.062685,-0.359002,-0.340847,-0.746894,0.994915,0.0
3,-18.437500,-81.201581,-142.885515,-147.982396,-0.112922,-0.200496,-5.5,28.1694,0.041776,40.50,0.000000,-18.750000,0.000000,-39.0625,-104.639356,-72.354762,-0.516340,1.086734,-0.979968,-15.787284,0.062500,0.042857,0.000000,3.267942,2.393803,4.031617,2.857143,0.206968,-0.206968,-69.127028,-79.641768,-0.029068,0.029068,-53.184942,-49.130807,0.022443,-0.022443,-59.918687,-61.226017,0.348109,-1.857504,-0.448435,-0.472277,0.108081,-0.364828,0.125000,0.465951,0.348803,-0.337480,0.976177,0.250000,1.010666,1.0
4,-20.569853,4.528548,8.305406,7.720009,0.008654,0.013824,-15.0,-8.8956,-0.012147,-14.75,5.514706,15.073529,1.666667,39.0625,5.458870,-4.577046,0.143089,-0.715168,0.062771,-192.035981,0.161765,0.166667,0.225000,-11.922309,-13.013223,-10.903010,9.583333,-0.396099,0.396099,-4.006798,10.817846,-0.108456,0.108456,-7.387436,0.223939,-0.159550,0.159550,-5.913708,3.336551,-0.292734,-0.273395,-0.397914,-0.207933,0.101515,0.067341,0.212463,0.268190,0.101214,0.220104,-0.132926,0.433073,1.032489,1.0


In [16]:
# HRV_S
# del data['HRV_S']
# del data['HRV_HF']
# data = data.reset_index()
# np.any(np.isnan(np.array(data)))
# col_name = data.columns.to_series()[np.isnan(data).any()]

# del data['HRV_HFn']
# del data['HRV_LnHF']
# np.isinf(data).values.sum()

In [17]:
print(np.isinf(data).values.sum())
print(data.columns.to_series()[np.isnan(data).any()])

0
Series([], dtype: object)


In [18]:
X_train, X_test, y_train, y_test = train_test_split(data, labels['COND_BIN'], test_size=0.2)

In [19]:
# defining parameter range
#Setting C: C is 1 by default and it’s a reasonable default choice. If you have a lot of noisy observations you should decrease it:
# it takes forever for 'linear' kernel+big values of C so not going to try it 
#One is advised to use GridSearchCV with C and gamma spaced exponentially far apart to choose good values.


param_grid = {'C': [0.001,0.1, 1, 10, 100], 
              'gamma': [1, 0.1, 0.01, 0.001,'scale','auto'],
              'kernel': ['rbf','sigmoid'],
              'class_weight':['balanced',None]} 


grid = GridSearchCV((SVC()), param_grid, cv=5, verbose = 3,n_jobs=-1)
grid.fit(X_train.to_numpy(), y_train.to_numpy())
res = grid.best_estimator_


Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END C=0.001, class_weight=balanced, gamma=1, kernel=sigmoid;, score=0.504 total time=   3.7s[CV 2/5] END C=0.001, class_weight=balanced, gamma=1, kernel=rbf;, score=0.503 total time=   3.8s

[CV 3/5] END C=0.001, class_weight=balanced, gamma=1, kernel=sigmoid;, score=0.497 total time=   3.7s
[CV 3/5] END C=0.001, class_weight=balanced, gamma=1, kernel=rbf;, score=0.497 total time=   3.8s[CV 1/5] END C=0.001, class_weight=balanced, gamma=1, kernel=rbf;, score=0.504 total time=   3.9s

[CV 5/5] END C=0.001, class_weight=balanced, gamma=1, kernel=rbf;, score=0.497 total time=   3.9s
[CV 2/5] END C=0.001, class_weight=balanced, gamma=1, kernel=sigmoid;, score=0.503 total time=   3.8s
[CV 4/5] END C=0.001, class_weight=balanced, gamma=1, kernel=sigmoid;, score=0.497 total time=   3.9s
[CV 2/5] END C=0.001, class_weight=balanced, gamma=0.1, kernel=rbf;, score=0.503 total time=   3.8s
[CV 4/5] END C=0.001, class_weight=ba

In [20]:
y_pred_SVM = grid.predict(X_test.to_numpy())
acc=accuracy_score(y_test.to_numpy(), y_pred_SVM)

print("Results\n\tAccuracy using SVM: {}".format(acc))

Results
	Accuracy using SVM: 0.49271402550091076


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels['AROUSAL_CAT'], test_size=0.2)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam, SGD,RMSprop


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.layers import LSTM, CuDNNLSTM
from keras.layers.embeddings import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
data.shape[1]

In [ ]:
def network(hp):

    
    model = Sequential()
    model.add(Dense(
        hp.Int('input_units', 256, 1024, 256),
        activation='relu',
        input_shape=(data.shape[1],)))
    for i in range(hp.Int('n_layers',1,6)):
        model.add(Dense(
            hp.Int(f'{i}_layer_units',  256, 1024, 256),
            activation='relu',#hp.Choice("activation", ["relu", "tanh"]), 
            ))
    if hp.Boolean("dropout"):
        model.add(Dropout(rate=0.25))
    
    model.add(Dense(1, activation='sigmoid'))
    
    learning_rate = hp.Choice("lr", values=[1e-1, 1e-2, 1e-3])
    adam = Adam(learning_rate=learning_rate)
#     sgd = SGD(learning_rate=learning_rate)
#     rms = RMSprop(learning_rate=learning_rate)  
    
    model.compile(
        loss='binary_crossentropy', 
        optimizer=adam,#keras.optimizers.Adam(learning_rate=learning_rate),
        metrics = ['accuracy'])
    return model


In [ ]:
# from tensorflow import keras
from kerastuner.tuners import RandomSearch, Hyperband
from kerastuner.engine.hyperparameters import HyperParameters
import time


early_stopping = EarlyStopping(
    monitor="val_accuracy",
    patience=10,
    restore_best_weights=True)

tuner = RandomSearch(
    network,
    objective = 'val_accuracy',
    max_trials = 50,
    executions_per_trial = 1,
    directory = os.path.normpath('C:/keras_tuner')
)

tuner.search(x=X_train,
            y=y_train,
            epochs=30,
            batch_size=64,
            callbacks=[early_stopping],
            validation_data=(X_test, y_test)
            )

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hp)

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', patience=50),
         ModelCheckpoint(filepath='best_model_NN.h5', monitor='val_accuracy', save_best_only=True)]

history = model.fit(X_train, y_train, epochs = 100, batch_size = 32,validation_data=(X_test, y_test),callbacks=callbacks)